## This Script is used to get the unique sequences from GraphAligner and measure the edit distance between the sequences from Bandage and GraphAligner

In [ ]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict
import gfapy

In [ ]:
# Open the GFA file
file_path = "Real.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [3]:
data_graph = []

In [4]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

             Name                                           Sequence
0               7  AACTCAAGTTTCGCCTAGTTCACAATACGAAGGCGCAGTAACATAC...
1               9  GCTAACACTCCCCACCCATGATTCGACTTGAAACCATCCTGACCCC...
2              11  GCCTCAACGTCGTCCACCGCCTGCACGAAGGCGAACTGGCCAAGGC...
3              13  TACTGAAGATATATCTGATATTTCTACATAGAAATAAGGTTTTAAT...
4              19  AAGTGCGCGCAGACACAAATCTCGCGATTGAAAAATTTTATTTTTC...
...           ...                                                ...
3852221  87719752  GGGAATGATGATTTCGCTGCCCCGCGGCAGATCCAGCGCCAGCAGG...
3852222  87719758  GGCGCCGTGCTCGAGCGAGTGCACCGCGTTCTCGTTCTGCTGCGGC...
3852223  87719770  CAGGGCGCTCTTGCCGCGGCTGGAGTTGCCGACGCCGTAGAACATC...
3852224  87719778  CGGTCAAAGAACCCGCGCAGGAATTCGGCAACGTGTTCGAATTCGA...
3852225  87719794  CGTCACCCTGATCTGCGAACAGCCCAAGGTGAAGCCGCACCGTCAG...

[3852226 rows x 2 columns]


In [5]:
df_graph['Sequence']

0          AACTCAAGTTTCGCCTAGTTCACAATACGAAGGCGCAGTAACATAC...
1          GCTAACACTCCCCACCCATGATTCGACTTGAAACCATCCTGACCCC...
2          GCCTCAACGTCGTCCACCGCCTGCACGAAGGCGAACTGGCCAAGGC...
3          TACTGAAGATATATCTGATATTTCTACATAGAAATAAGGTTTTAAT...
4          AAGTGCGCGCAGACACAAATCTCGCGATTGAAAAATTTTATTTTTC...
                                 ...                        
3852221    GGGAATGATGATTTCGCTGCCCCGCGGCAGATCCAGCGCCAGCAGG...
3852222    GGCGCCGTGCTCGAGCGAGTGCACCGCGTTCTCGTTCTGCTGCGGC...
3852223    CAGGGCGCTCTTGCCGCGGCTGGAGTTGCCGACGCCGTAGAACATC...
3852224    CGGTCAAAGAACCCGCGCAGGAATTCGGCAACGTGTTCGAATTCGA...
3852225    CGTCACCCTGATCTGCGAACAGCCCAAGGTGAAGCCGCACCGTCAG...
Name: Sequence, Length: 3852226, dtype: object

In [6]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [7]:
df_graph

,Name,Sequence
0,7,AACTCAAGTTTCGCCTAGTTCACAATACGAAGGCGCAGTAACATAC...
1,9,GCTAACACTCCCCACCCATGATTCGACTTGAAACCATCCTGACCCC...
2,11,GCCTCAACGTCGTCCACCGCCTGCACGAAGGCGAACTGGCCAAGGC...
3,13,TACTGAAGATATATCTGATATTTCTACATAGAAATAAGGTTTTAAT...
4,19,AAGTGCGCGCAGACACAAATCTCGCGATTGAAAAATTTTATTTTTC...
...,...,...
3852221,87719752,GGGAATGATGATTTCGCTGCCCCGCGGCAGATCCAGCGCCAGCAGG...
3852222,87719758,GGCGCCGTGCTCGAGCGAGTGCACCGCGTTCTCGTTCTGCTGCGGC...
3852223,87719770,CAGGGCGCTCTTGCCGCGGCTGGAGTTGCCGACGCCGTAGAACATC...
3852224,87719778,CGGTCAAAGAACCCGCGCAGGAATTCGGCAACGTGTTCGAATTCGA...


In [30]:
df_GraphAligner_Paths=pd.read_csv('Real_Results/SequenceBandageSPAlignerGraphAligner.tsv', sep='\t')

In [31]:
df_GraphAligner_Paths

,Unnamed: 0,Query,Sequence,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Sequence_GraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,NaN
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,61,858,26862905,Different,NaN
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,NaN
3,3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...,64,588,81232484,Full,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
4,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,NaN
...,...,...,...,...,...,...,...,...,...
136,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,180,959,"6903442, 506744",Different,NaN
137,137,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,130,990,605844,Full,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...
138,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,72,872,"32592258, 758212",Different,NaN
139,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,142,2115,"4764923, 2707722, 11583809, 43494081, 43494077...",Different,NaN


In [32]:
selected_rows_BandageVSGraphAligner = df_GraphAligner_Paths[df_GraphAligner_Paths['FinalResultBandageVSGraphAligner'] != 'Full']
selected_rows_BandageVSGraphAligner.columns

Index(['Unnamed: 0', 'Query', 'Sequence', 'Sequence_Bandage',
       'Start_GraphAligner', 'End_GraphAligner', 'Path_GraphAligner',
       'FinalResultBandageVSGraphAligner', 'Sequence_GraphAligner'],
      dtype='object')

In [33]:
df_BandageVSGraphAligner=selected_rows_BandageVSGraphAligner[['Query', 'Sequence', 'Sequence_Bandage',
       'Start_GraphAligner', 'End_GraphAligner', 'Path_GraphAligner',
       'FinalResultBandageVSGraphAligner']].copy()


In [34]:
df_BandageVSGraphAligner.reset_index()

,index,Query,Sequence,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,61,858,26862905,Different
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different
3,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different
4,5,gb|AB901141|+|0-864|ARO:3002353|GES-24,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different
...,...,...,...,...,...,...,...,...
103,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different
104,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,180,959,"6903442, 506744",Different
105,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,72,872,"32592258, 758212",Different
106,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,142,2115,"4764923, 2707722, 11583809, 43494081, 43494077...",Different


In [35]:
df_BandageVSGraphAligner.to_csv('Real_Results/BandageVSGraphAlignerSequences.tsv',sep='\t')

In [36]:
df_GraphAligner_Path= pd.read_csv('Real_Results/BandageVSGraphAlignerSequences.tsv', sep='\t')

In [37]:
df_GraphAligner_Path

,Unnamed: 0,Query,Sequence,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,61,858,26862905,Different
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different
3,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different
4,5,gb|AB901141|+|0-864|ARO:3002353|GES-24,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different
...,...,...,...,...,...,...,...,...
103,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different
104,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,180,959,"6903442, 506744",Different
105,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,72,872,"32592258, 758212",Different
106,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,142,2115,"4764923, 2707722, 11583809, 43494081, 43494077...",Different


In [41]:
df_GraphAligner_Path[['Path1', 'Path2','Path3','Path4','Path5','Path6','Path7','Path8','Path9','Path10']] = df_GraphAligner_Path['Path_GraphAligner'].str.split(',', expand=True)

In [42]:
df_GraphAligner_Path

,Unnamed: 0,Query,Sequence,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,Path3,Path4,Path5,Path6,Path7,Path8,Path9,Path10
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,61,858,26862905,Different,26862905,None,None,None,None,None,None,None,None,None
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
3,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,2638631,6283287,5689755,None,None,None,None,None,None
4,5,gb|AB901141|+|0-864|ARO:3002353|GES-24,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,15299651,77161691,15936737,31689609,31674241,385047,None,None,None
104,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,180,959,"6903442, 506744",Different,6903442,506744,None,None,None,None,None,None,None,None
105,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,72,872,"32592258, 758212",Different,32592258,758212,None,None,None,None,None,None,None,None
106,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,142,2115,"4764923, 2707722, 11583809, 43494081, 43494077...",Different,4764923,2707722,11583809,43494081,43494077,3922650,None,None,None,None


## Get the rows that have a unique node, Path1

In [43]:
# Get the rows where Path 2 is  None
Path1_Only = df_GraphAligner_Path[df_GraphAligner_Path['Path2'].isna()]

# Print the resulting dataframe
Path1_Only 

,Unnamed: 0,Query,Sequence,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,Path3,Path4,Path5,Path6,Path7,Path8,Path9,Path10
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
1,1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...,61,858,26862905,Different,26862905,None,None,None,None,None,None,None,None,None
2,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
4,5,gb|AB901141|+|0-864|ARO:3002353|GES-24,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
8,12,gb|AF099140|+|136-1363|ARO:3002826|EreA2,ATGGACACAACGCAGGTCACATTGATACACAAAATTCTAGCTGCGG...,ATGGACACAACGCAGGTCACATTGATACACAAAATTCTAGCTGCGG...,128,1345,31482731,Different,31482731,None,None,None,None,None,None,None,None,None
9,13,gb|AF118110.1|-|71-1037|ARO:3003002|CfxA2,ATGACATGGAGAACGACCAGAACACTTTTACAGCCTCAAAAGCTGG...,ATGACATGGAGAACGACCAGAACACTTTTACAGCCTCAAAAGCTGG...,142,1107,5661688,Different,5661688,None,None,None,None,None,None,None,None,None
11,15,gb|AF137361|+|63-852|ARO:3002605|aadA5,ATGAACGTACGTAAACACAAGGCTAGTTTTTTTAGCGTAGTAATTA...,ATGAACGTACGTAAACACAAGGCTAGTTTTTTTAGCGTAGTAATTA...,57,845,1370533,Different,1370533,None,None,None,None,None,None,None,None,None
12,16,gb|AF156486|+|1331-2195|ARO:3002330|GES-1,ATGGGTGAATTTTTCCCTGCACAAGTTTTCAAGCAGCTGTCCCACG...,ATGGGTGAATTTTTCCCTGCACAAGTTTTCAAGCAGCTGTCCCACG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
19,26,gb|AF326355|+|0-864|ARO:3002331|GES-2,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None
20,27,gb|AF329699|+|372-1236|ARO:3002337|GES-8,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None


In [45]:
#Drop the 0 parts
Path1_Only = Path1_Only.drop(Path1_Only[(Path1_Only['Start_GraphAligner'] == 0) & (Path1_Only['End_GraphAligner'] == 0)].index)


In [46]:
# Get the sequence where we have only Path1
merged_df_Path1= pd.merge(Path1_Only , df_graph, left_on='Path1', right_on='Name')
merged_df_Path1

,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,Path3,Path4,Path5,Path6,Path7,Path8,Path9,Path10,Name,Sequence_y
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
1,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
2,5,gb|AB901141|+|0-864|ARO:3002353|GES-24,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
3,16,gb|AF156486|+|1331-2195|ARO:3002330|GES-1,ATGGGTGAATTTTTCCCTGCACAAGTTTTCAAGCAGCTGTCCCACG...,ATGGGTGAATTTTTCCCTGCACAAGTTTTCAAGCAGCTGTCCCACG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
4,26,gb|AF326355|+|0-864|ARO:3002331|GES-2,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
5,27,gb|AF329699|+|372-1236|ARO:3002337|GES-8,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
6,51,gb|AY260546|+|4477-5341|ARO:3002336|GES-7,ATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,ATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
7,53,gb|AY494717|+|0-864|ARO:3002334|GES-5,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
8,54,gb|AY494718|+|0-864|ARO:3002335|GES-6,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...
9,58,gb|AY920928|+|2689-3553|ARO:3002338|GES-9,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...,93,956,32422725,Different,32422725,None,None,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...


In [47]:
merged_df_Path1.dtypes

Unnamed: 0                           int64
Query                               object
Sequence_x                          object
Sequence_Bandage                    object
Start_GraphAligner                  object
End_GraphAligner                    object
Path_GraphAligner                   object
FinalResultBandageVSGraphAligner    object
Path1                               object
Path2                               object
Path3                               object
Path4                               object
Path5                               object
Path6                               object
Path7                               object
Path8                               object
Path9                               object
Path10                              object
Name                                object
Sequence_y                          object
dtype: object

In [48]:
merged_df_Path1['Start_GraphAligner']=merged_df_Path1['Start_GraphAligner'].astype(int)
merged_df_Path1['End_GraphAligner']=merged_df_Path1['End_GraphAligner'].astype(int)

In [49]:
merged_df_Path1.dtypes

Unnamed: 0                           int64
Query                               object
Sequence_x                          object
Sequence_Bandage                    object
Start_GraphAligner                   int64
End_GraphAligner                     int64
Path_GraphAligner                   object
FinalResultBandageVSGraphAligner    object
Path1                               object
Path2                               object
Path3                               object
Path4                               object
Path5                               object
Path6                               object
Path7                               object
Path8                               object
Path9                               object
Path10                              object
Name                                object
Sequence_y                          object
dtype: object

In [ ]:
merged_df_Path1

In [50]:
def extract_sequences_Path1(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start_GraphAligner']
        end_pos = row['End_GraphAligner']
        sequence = row['Sequence_y']
        
        extracted_sequence = sequence[start_pos-1:end_pos]
        dataframe.loc[index, 'Result_node1'] = extracted_sequence
    
    return dataframe

In [52]:
extracted_sequences_Path1_df = extract_sequences_Path1(merged_df_Path1)
extracted_sequences_Path1_df
#Save the dataframe to a pdf
extracted_sequences_Path1_df.to_csv("Real_Results/Path1_TestSequences.tsv", sep="\t")

In [53]:
extracted_sequences_Path1_df

,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,...,Path4,Path5,Path6,Path7,Path8,Path9,Path10,Name,Sequence_y,Result_node1
0,0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
1,2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
2,5,gb|AB901141|+|0-864|ARO:3002353|GES-24,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
3,16,gb|AF156486|+|1331-2195|ARO:3002330|GES-1,ATGGGTGAATTTTTCCCTGCACAAGTTTTCAAGCAGCTGTCCCACG...,ATGGGTGAATTTTTCCCTGCACAAGTTTTCAAGCAGCTGTCCCACG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
4,26,gb|AF326355|+|0-864|ARO:3002331|GES-2,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
5,27,gb|AF329699|+|372-1236|ARO:3002337|GES-8,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
6,51,gb|AY260546|+|4477-5341|ARO:3002336|GES-7,ATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,ATGAGGGTAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
7,53,gb|AY494717|+|0-864|ARO:3002334|GES-5,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
8,54,gb|AY494718|+|0-864|ARO:3002335|GES-6,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...
9,58,gb|AY920928|+|2689-3553|ARO:3002338|GES-9,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...,93,956,32422725,Different,32422725,None,...,None,None,None,None,None,None,None,32422725,ACGAATTGTTAGACGGGCGTCAACTATTTGTCCGTGCTCAGGATGA...,TACGGCCGATAGTTTCGGGGCCGTTGTATACACCGCTACAGCGTAA...


In [54]:
import Levenshtein
#Create a method to calculate the levenstein distance
def calculate_edit_distance_sequence_GraphAligner(row):
    return Levenshtein.distance(row['Sequence_x'], row['Result_node1'])

In [ ]:
extracted_sequences_Path1_df['edit_distance_query_sequence_GraphAligner'] = extracted_sequences_Path1_df.apply(calculate_edit_distance_sequence_GraphAligner, axis=1)

In [ ]:
extracted_sequences_Path1_df

In [ ]:
extracted_sequences_Path1_df['sequence_length'] = extracted_sequences_Path1_df['Sequence_x'].apply(len)
extracted_sequences_Path1_df['EditDistance_GraphAligner_Query']=1-(extracted_sequences_Path1_df['edit_distance_query_sequence_GraphAligner']/extracted_sequences_Path1_df['sequence_length'])

In [ ]:
extracted_sequences_Path1_df

In [55]:
extracted_sequences_Path1_df.to_csv('Real_Results/GraphAlignerSequencesEditDistancePath1.tsv',sep='\t')

In [56]:
# Get the rows where Path 2 is not None
df_multiple = df_GraphAligner_Path[df_GraphAligner_Path['Path2'].notna() ]

# Print the resulting dataframe
df_multiple

,Unnamed: 0,Query,Sequence,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,Path3,Path4,Path5,Path6,Path7,Path8,Path9,Path10
3,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,2638631,6283287,5689755,None,None,None,None,None,None
5,8,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,60915537,2728545,None,None,None,None,None,None
6,9,gb|AF043381|+|943-1744|ARO:3001414|OXA-19,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,72,872,"32592258, 758212",Different,32592258,758212,None,None,None,None,None,None,None,None
7,10,gb|AF047479|+|1295-2087|ARO:3002603|aadA3,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...,"13, 13","804, 801","5638634, 23556688, 66940499, 66940495, 3661600...",Different,5638634,23556688,66940499,66940495,3661600,385047 5638634,23556688,66940499,81873962,31582859
10,14,gb|AF135373|+|11-908|ARO:3002244|CARB-5,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...,146,1042,"267426, 63777837",Different,267426,63777837,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,15299651,77161691,15936737,31689609,31674241,385047,None,None,None
104,136,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,180,959,"6903442, 506744",Different,6903442,506744,None,None,None,None,None,None,None,None
105,138,gb|X75562|+|134-935|ARO:3001402|OXA-7,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...,72,872,"32592258, 758212",Different,32592258,758212,None,None,None,None,None,None,None,None
106,139,gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,142,2115,"4764923, 2707722, 11583809, 43494081, 43494077...",Different,4764923,2707722,11583809,43494081,43494077,3922650,None,None,None,None


In [57]:
merged_df_multiple_Path1= pd.merge(df_multiple, df_graph, left_on='Path1', right_on='Name')
merged_df_multiple_Path1


,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,Path3,Path4,Path5,Path6,Path7,Path8,Path9,Path10,Name,Sequence_y
0,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,2638631,6283287,5689755,None,None,None,None,None,None,50056,TTATTGGGACTTACATTTATTTTGAGTGGTGTATTACCTGCAAGTT...
1,8,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,60915537,2728545,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...
2,132,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,60915537,2728545,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...
3,9,gb|AF043381|+|943-1744|ARO:3001414|OXA-19,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,72,872,"32592258, 758212",Different,32592258,758212,None,None,None,None,None,None,None,None,32592258,CACGAGCTAAGTCATTGGTAGCGCAGGATTTACTGCTACTTTTACA...
4,20,gb|AF231133|+|1345-2146|ARO:3001423|OXA-28,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,72,872,"32592258, 758212",Different,32592258,758212,None,None,None,None,None,None,None,None,32592258,CACGAGCTAAGTCATTGGTAGCGCAGGATTTACTGCTACTTTTACA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,119,gb|LN873256.1|-|3238-4027|ARO:3004682|aadA27,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,172,960,"96908, 293994",Different,96908,293994,None,None,None,None,None,None,None,None,96908,AGACAAAACCCCTCTAAGTTGCAAGACTCAGAGGGGTTTTTCTTTA...
69,121,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,254,2173,"4147608, 3205126, 17316950",Different,4147608,3205126,17316950,None,None,None,None,None,None,None,4147608,TAAATATGCGGCAAGGTATTCTTAAATAAACTGTCAATTTGATAGT...
70,125,gb|M72415.1|+|1120-1984|ARO:3004683|aadS,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,3254,4117,"18574655, 224504",Different,18574655,224504,None,None,None,None,None,None,None,None,18574655,CTTGTCAGAAGAAGAACTCTTACATCTTCGTTTTTCTCCGACCATT...
71,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,15299651,77161691,15936737,31689609,31674241,385047,None,None,None,31214326,AACTTTGTTTTAGGGCGACTGCCCTGCTGCGTAACATCGTTGCTGC...


In [58]:
merged_df_multiple_Path1['Path2'] = merged_df_multiple_Path1['Path2'].str.strip()


In [59]:
merged_df_multiple_Path2=pd.merge(merged_df_multiple_Path1, df_graph,left_on='Path2', right_on='Name')
merged_df_multiple_Path2

,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,...,Path5,Path6,Path7,Path8,Path9,Path10,Name_x,Sequence_y,Name_y,Sequence
0,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,2638631,...,None,None,None,None,None,None,50056,TTATTGGGACTTACATTTATTTTGAGTGGTGTATTACCTGCAAGTT...,2638631,TGGATAATGATGCCTTATGGCCGAATTTTACAAAATTCATTCACGG...
1,8,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,...,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
2,132,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,...,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
3,62,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,65,664,"36782492, 3274129, 60915537, 2728545",Different,36782492,3274129,...,None,None,None,None,None,None,36782492,CGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGATGCCTAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
4,9,gb|AF043381|+|943-1744|ARO:3001414|OXA-19,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,72,872,"32592258, 758212",Different,32592258,758212,...,None,None,None,None,None,None,32592258,CACGAGCTAAGTCATTGGTAGCGCAGGATTTACTGCTACTTTTACA...,758212,TTAGCCACCAATGATGCCCTCACTTTCCATGATTTTGGTGGGAATG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,119,gb|LN873256.1|-|3238-4027|ARO:3004682|aadA27,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,172,960,"96908, 293994",Different,96908,293994,...,None,None,None,None,None,None,96908,AGACAAAACCCCTCTAAGTTGCAAGACTCAGAGGGGTTTTTCTTTA...,293994,GTCGCTGCATCAACCCATTTTGCAGCGCATGATCCAAGAATATAAA...
69,121,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,254,2173,"4147608, 3205126, 17316950",Different,4147608,3205126,...,None,None,None,None,None,None,4147608,TAAATATGCGGCAAGGTATTCTTAAATAAACTGTCAATTTGATAGT...,3205126,TCTTTTTTGGGGAATGTGCAGATGGAAGTCATTTGTGCCATCCTTG...
70,125,gb|M72415.1|+|1120-1984|ARO:3004683|aadS,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,3254,4117,"18574655, 224504",Different,18574655,224504,...,None,None,None,None,None,None,18574655,CTTGTCAGAAGAAGAACTCTTACATCTTCGTTTTTCTCCGACCATT...,224504,CAATTATAGAATGGTCGGAGAAAAACGAAGATGTAAGAGTTCTTCT...
71,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,15299651,...,31689609,31674241,385047,None,None,None,31214326,AACTTTGTTTTAGGGCGACTGCCCTGCTGCGTAACATCGTTGCTGC...,15299651,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...


In [60]:
#Rename Name_X, Sequence_X, Name_y, Sequence_y
merged_df_multiple_Path2.rename(columns = {'Sequence':'Sequence_Path1'}, inplace = True)
merged_df_multiple_Path2.rename(columns = {'Sequence_y':'Sequence_Path2'}, inplace = True)
merged_df_multiple_Path2

,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,...,Path5,Path6,Path7,Path8,Path9,Path10,Name_x,Sequence_Path2,Name_y,Sequence_Path1
0,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,2638631,...,None,None,None,None,None,None,50056,TTATTGGGACTTACATTTATTTTGAGTGGTGTATTACCTGCAAGTT...,2638631,TGGATAATGATGCCTTATGGCCGAATTTTACAAAATTCATTCACGG...
1,8,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,...,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
2,132,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,...,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
3,62,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,65,664,"36782492, 3274129, 60915537, 2728545",Different,36782492,3274129,...,None,None,None,None,None,None,36782492,CGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGATGCCTAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
4,9,gb|AF043381|+|943-1744|ARO:3001414|OXA-19,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,72,872,"32592258, 758212",Different,32592258,758212,...,None,None,None,None,None,None,32592258,CACGAGCTAAGTCATTGGTAGCGCAGGATTTACTGCTACTTTTACA...,758212,TTAGCCACCAATGATGCCCTCACTTTCCATGATTTTGGTGGGAATG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,119,gb|LN873256.1|-|3238-4027|ARO:3004682|aadA27,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,172,960,"96908, 293994",Different,96908,293994,...,None,None,None,None,None,None,96908,AGACAAAACCCCTCTAAGTTGCAAGACTCAGAGGGGTTTTTCTTTA...,293994,GTCGCTGCATCAACCCATTTTGCAGCGCATGATCCAAGAATATAAA...
69,121,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,254,2173,"4147608, 3205126, 17316950",Different,4147608,3205126,...,None,None,None,None,None,None,4147608,TAAATATGCGGCAAGGTATTCTTAAATAAACTGTCAATTTGATAGT...,3205126,TCTTTTTTGGGGAATGTGCAGATGGAAGTCATTTGTGCCATCCTTG...
70,125,gb|M72415.1|+|1120-1984|ARO:3004683|aadS,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,3254,4117,"18574655, 224504",Different,18574655,224504,...,None,None,None,None,None,None,18574655,CTTGTCAGAAGAAGAACTCTTACATCTTCGTTTTTCTCCGACCATT...,224504,CAATTATAGAATGGTCGGAGAAAAACGAAGATGTAAGAGTTCTTCT...
71,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,15299651,...,31689609,31674241,385047,None,None,None,31214326,AACTTTGTTTTAGGGCGACTGCCCTGCTGCGTAACATCGTTGCTGC...,15299651,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...


In [61]:
merged_df_multiple_Path2.dtypes

Unnamed: 0                           int64
Query                               object
Sequence_x                          object
Sequence_Bandage                    object
Start_GraphAligner                  object
End_GraphAligner                    object
Path_GraphAligner                   object
FinalResultBandageVSGraphAligner    object
Path1                               object
Path2                               object
Path3                               object
Path4                               object
Path5                               object
Path6                               object
Path7                               object
Path8                               object
Path9                               object
Path10                              object
Name_x                              object
Sequence_Path2                      object
Name_y                              object
Sequence_Path1                      object
dtype: object

In [64]:
merged_df_multiple_Path2

,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,Path2,...,Path5,Path6,Path7,Path8,Path9,Path10,Name_x,Sequence_Path2,Name_y,Sequence_Path1
0,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,2638631,...,None,None,None,None,None,None,50056,TTATTGGGACTTACATTTATTTTGAGTGGTGTATTACCTGCAAGTT...,2638631,TGGATAATGATGCCTTATGGCCGAATTTTACAAAATTCATTCACGG...
1,8,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,...,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
2,132,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,3274129,...,None,None,None,None,None,None,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
3,62,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,65,664,"36782492, 3274129, 60915537, 2728545",Different,36782492,3274129,...,None,None,None,None,None,None,36782492,CGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGATGCCTAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
4,9,gb|AF043381|+|943-1744|ARO:3001414|OXA-19,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,72,872,"32592258, 758212",Different,32592258,758212,...,None,None,None,None,None,None,32592258,CACGAGCTAAGTCATTGGTAGCGCAGGATTTACTGCTACTTTTACA...,758212,TTAGCCACCAATGATGCCCTCACTTTCCATGATTTTGGTGGGAATG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,119,gb|LN873256.1|-|3238-4027|ARO:3004682|aadA27,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,172,960,"96908, 293994",Different,96908,293994,...,None,None,None,None,None,None,96908,AGACAAAACCCCTCTAAGTTGCAAGACTCAGAGGGGTTTTTCTTTA...,293994,GTCGCTGCATCAACCCATTTTGCAGCGCATGATCCAAGAATATAAA...
69,121,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,254,2173,"4147608, 3205126, 17316950",Different,4147608,3205126,...,None,None,None,None,None,None,4147608,TAAATATGCGGCAAGGTATTCTTAAATAAACTGTCAATTTGATAGT...,3205126,TCTTTTTTGGGGAATGTGCAGATGGAAGTCATTTGTGCCATCCTTG...
70,125,gb|M72415.1|+|1120-1984|ARO:3004683|aadS,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,3254,4117,"18574655, 224504",Different,18574655,224504,...,None,None,None,None,None,None,18574655,CTTGTCAGAAGAAGAACTCTTACATCTTCGTTTTTCTCCGACCATT...,224504,CAATTATAGAATGGTCGGAGAAAAACGAAGATGTAAGAGTTCTTCT...
71,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,15299651,...,31689609,31674241,385047,None,None,None,31214326,AACTTTGTTTTAGGGCGACTGCCCTGCTGCGTAACATCGTTGCTGC...,15299651,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...


In [65]:
merged_df_multiple_Path2.to_csv('merged_df_multiple_Path2.tsv',sep='\t')

In [62]:
def extract_sequences_multiple_sequences(dataframe):
    for index, row in dataframe.iterrows():
        start_pos = row['Start_GraphAligner']
        sequence_1 = row['Sequence_Path2']
       
        #Get the first sequence, and remove 55 kmer from it
        extracted_sequence_first = sequence_1[start_pos-1:]
        
        dataframe.loc[index, 'Sequencefirst'] = extracted_sequence_first
    
    return dataframe

In [66]:
merged_df_multiple_Path2_modified=pd.read_csv('merged_df_multiple_Path2.tsv', sep='\t')

In [68]:
merged_df_multiple_Path2_modified

,Unnamed: 0.1,Unnamed: 0,Query,Sequence_x,Sequence_Bandage,Start_GraphAligner,End_GraphAligner,Path_GraphAligner,FinalResultBandageVSGraphAligner,Path1,...,Path5,Path6,Path7,Path8,Path9,Path10,Name_x,Sequence_Path2,Name_y,Sequence_Path1
0,0,4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...,353,1237,"50056, 2638631, 6283287, 5689755",Different,50056,...,NaN,NaN,NaN,NaN,NaN,NaN,50056,TTATTGGGACTTACATTTATTTTGAGTGGTGTATTACCTGCAAGTT...,2638631,TGGATAATGATGCCTTATGGCCGAATTTTACAAAATTCATTCACGG...
1,1,8,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,TATCGCGTGTCTTTCGAGTACGGCATTAGCTGGTTCAATTACAGAA...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,...,NaN,NaN,NaN,NaN,NaN,NaN,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
2,2,132,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,ATGAGCGATATGCACTCGCTGCTGATAGCGGCAATATTGGGTGTGG...,14,627,"24065316, 3274129, 60915537, 2728545",Different,24065316,...,NaN,NaN,NaN,NaN,NaN,NaN,24065316,TGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAAAACAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
3,3,62,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,ATGAGGGAAGCGGTGACCATCGAAATTTCGAACCAACTATCAGAGG...,65,664,"36782492, 3274129, 60915537, 2728545",Different,36782492,...,NaN,NaN,NaN,NaN,NaN,NaN,36782492,CGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGATGCCTAA...,3274129,AGTGTGACGGAATCGTTGCTGTTGGTCACGATGCTGTACTTTGTGA...
4,4,9,gb|AF043381|+|943-1744|ARO:3001414|OXA-19,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,ATGTTACGCAGCAGCAGTCGCCCTAAAACAAAGTTAGGCATCACAA...,72,872,"32592258, 758212",Different,32592258,...,NaN,NaN,NaN,NaN,NaN,NaN,32592258,CACGAGCTAAGTCATTGGTAGCGCAGGATTTACTGCTACTTTTACA...,758212,TTAGCCACCAATGATGCCCTCACTTTCCATGATTTTGGTGGGAATG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,119,gb|LN873256.1|-|3238-4027|ARO:3004682|aadA27,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,ATGAATATTAAACCTGCATCAGAAGCTTCACTCAAAGATTGGTTAG...,172,960,"96908, 293994",Different,96908,...,NaN,NaN,NaN,NaN,NaN,NaN,96908,AGACAAAACCCCTCTAAGTTGCAAGACTCAGAGGGGTTTTTCTTTA...,293994,GTCGCTGCATCAACCCATTTTGCAGCGCATGATCCAAGAATATAAA...
69,69,121,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,254,2173,"4147608, 3205126, 17316950",Different,4147608,...,NaN,NaN,NaN,NaN,NaN,NaN,4147608,TAAATATGCGGCAAGGTATTCTTAAATAAACTGTCAATTTGATAGT...,3205126,TCTTTTTTGGGGAATGTGCAGATGGAAGTCATTTGTGCCATCCTTG...
70,70,125,gb|M72415.1|+|1120-1984|ARO:3004683|aadS,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,ATGCAATTCACAAAGATTGATATAAATAATTGGACACGAAAAGAGT...,3254,4117,"18574655, 224504",Different,18574655,...,NaN,NaN,NaN,NaN,NaN,NaN,18574655,CTTGTCAGAAGAAGAACTCTTACATCTTCGTTTTTCTCCGACCATT...,224504,CAATTATAGAATGGTCGGAGAAAAACGAAGATGTAAGAGTTCTTCT...
71,71,135,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCAAAT...,3,967,"31214326, 15299651, 77161691, 15936737, 316896...",Different,31214326,...,31689609,31674241,385047.0,NaN,NaN,NaN,31214326,AACTTTGTTTTAGGGCGACTGCCCTGCTGCGTAACATCGTTGCTGC...,15299651,TGTTATGGAGCAGCAACGATGTTACGCAGCAGGGCAGTCGCCCTAA...


In [67]:
merged_df_multiple_Path2_modified['Start_GraphAligner']=merged_df_multiple_Path2['Start_GraphAligner'].astype(int)
merged_df_multiple_Path2_modified['End_GraphAligner']=merged_df_multiple_Path2['End_GraphAligner'].astype(int)

ValueError: invalid literal for int() with base 10: '13, 13'

In [ ]:
extracted_sequences_2 = extract_sequences_multiple_sequences(merged_df_multiple_Path2)
extracted_sequences_2.to_csv('TestSeque.tsv',sep='\t')

extracted_sequences_2

In [ ]:
def extract_sequences_multiple_sequences_path2(dataframe):
    for index, row in dataframe.iterrows():
        end_pos = row['End_GraphAligner_x']
        sequence_2 = row['Sequence_Path1']
       
        #Get the first sequence, and remove 55 kmer from it
        extracted_sequence_second = sequence_2[:end_pos]
        
        dataframe.loc[index, 'SequencefSecond'] = extracted_sequence_second
    
    return dataframe

In [ ]:
extracted_sequences_3 = extract_sequences_multiple_sequences_path2(extracted_sequences_2)
extracted_sequences_3.to_csv('TestSeque3.tsv',sep='\t')

extracted_sequences_3

In [ ]:
def extract_sequences_multiple_sequences_overlap(dataframe):
   
    for index, row in dataframe.iterrows():
        sequence_first = row['Sequencefirst']
        length_end=len(sequence_first)-55
       
        #Get the first sequence, and the last 55 characters
        extracted_sequence_first = sequence_first[:len(sequence_first)-55]
        
        dataframe.loc[index, 'length'] = len(sequence_first)
        dataframe.loc[index, 'length_end'] = length_end
        dataframe.loc[index, 'FinalSequence1'] = sequence_first[:length_end]
    
    return dataframe
    

In [ ]:
extracted_sequences_multiple = extract_sequences_multiple_sequences_overlap(extracted_sequences_3)

In [ ]:
extracted_sequences_multiple

In [ ]:
def extract_sequences_multiple_sequences_final(dataframe):
   
    for index, row in dataframe.iterrows():
        sequence_first = row['FinalSequence1']
        sequence_second = row['SequencefSecond']
       
        #Get the first sequence, and the last 55 characters
        extracted_sequence_final = sequence_first+sequence_second
        
      
        dataframe.loc[index, 'FinalSequence'] = sequence_first + sequence_second
    
    return dataframe

In [ ]:
extracted_sequences_multiple_final =extract_sequences_multiple_sequences_final(extracted_sequences_multiple)

In [ ]:
extracted_sequences_multiple_final

In [ ]:
import Levenshtein
#Create a method to calculate the levenstein distance
def calculate_edit_distance_sequence_GraphAligner_multiple(row):
    return Levenshtein.distance(row['Sequence_x'], row['FinalSequence'])

In [ ]:
extracted_sequences_multiple_final['edit_distance_query_sequence_GraphAligner'] =calculate_edit_distance_sequence_GraphAligner_multiple(extracted_sequences_multiple_final)

In [ ]:
extracted_sequences_multiple_final

In [ ]:
extracted_sequences_multiple_final['sequence_length'] = extracted_sequences_multiple_final['Sequence_x'].apply(len)
extracted_sequences_multiple_final['EditDistance_GraphAligner_Query']=1-(extracted_sequences_multiple_final['edit_distance_query_sequence_GraphAligner']/extracted_sequences_multiple_final['sequence_length'])

In [ ]:
#Drop some columns
extracted_sequences_multiple_final=extracted_sequences_multiple_final.drop(['Path_Bandage', 'Path_GraphAligner','Equal','Name_x','Name_y'], axis=1)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequence_Path1':'Sequence_Path1'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequence_Path2':'Sequence_FromGraph_Path2'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'Sequencefirst':'Sequence_FromGraph_Path1_No_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'length':'Length_Sequence_Path1_No_Overlap'}, inplace = True)


In [ ]:
extracted_sequences_multiple_final.rename(columns = {'length_end':'Length_Sequence_Path1_With_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'FinalSequence1':'Sequence_Path1_With_Overlap'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.rename(columns = {'FinalSequence':'Concatenated_Sequence'}, inplace = True)

In [ ]:
extracted_sequences_multiple_final.to_csv("CAMIH1_Results/MultiplePath_TestSequences.tsv",sep='\t')

## This section measures the edit distance between the sequences 

In [ ]:
import Levenshtein

In [ ]:
bandage_df= pd.read_csv('Real_Results/TestDistance.tsv', sep='\t')
bandage_df

In [ ]:
#Get the sequences from bandage where the Path2 in GraphAligner=none
distance_df=pd.merge(extracted_sequences_Path1_df,bandage_df, on='Query')

In [ ]:
distance_df

In [ ]:
distance_df.rename(columns = {'Result_node1':'Sequence_GraphAligner'}, inplace = True)
distance_df.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)

In [ ]:
distance_df=distance_df.drop('Unnamed: 0_x', axis=1)

In [ ]:
distance_df=distance_df.drop('Unnamed: 0_y', axis=1)

In [ ]:
distance_df

In [ ]:
#Create a method to calculate the levenstein distance
def calculate_edit_distance_Bandage_GraphAligner(row):
    return Levenshtein.distance(row['Sequence_Bandage'], row['Sequence_GraphAligner'])

In [ ]:
distance_df['edit_distance_query_Bandage_GraphAligner'] = distance_df.apply(calculate_edit_distance_Bandage_GraphAligner, axis=1)

In [ ]:
distance_df.dtypes

In [ ]:
type_counts_bandage_GraphAligner=distance_df['edit_distance_query_Bandage_GraphAligner'].value_counts().reset_index()

In [ ]:
# Rename the columns
type_counts_bandage_GraphAligner.columns = ['Type', 'Count']

# Display the result
print(type_counts_bandage_GraphAligner)

In [ ]:
type_counts_bandage_GraphAligner.to_csv("Real_Results/type_counts_bandage_GraphAligner.tsv",sep='\t')

In [ ]:
#Get the sequences from bandage where the Path2 in GraphAligner is not none
distance_df_multiple=pd.merge(extracted_sequences_multiple_final,bandage_df, on='Query')
distance_df_multiple

In [ ]:
distance_df_multiple.rename(columns = {'Concatenated_Sequence':'Sequence_GraphAligner'}, inplace = True)
distance_df_multiple.rename(columns = {'Sequence':'Sequence_Bandage'}, inplace = True)

In [ ]:
distance_df_multiple['edit_distance_query_Bandage_GraphAligner'] = distance_df_multiple.apply(calculate_edit_distance_Bandage_GraphAligner, axis=1)

In [ ]:
distance_df_multiple

In [ ]:
type_counts_bandage_GraphAligner_multiple=distance_df_multiple['edit_distance_query_Bandage_GraphAligner'].value_counts().reset_index()

In [ ]:
# Rename the columns
type_counts_bandage_GraphAligner_multiple.columns = ['Type', 'Count']

# Display the result
print(type_counts_bandage_GraphAligner_multiple)

In [ ]:
type_counts_bandage_GraphAligner_multiple.to_csv("Real_Results/type_counts_bandage_GraphAligner_multiple.tsv",sep='\t')

In [ ]:
#Read the query and sequences from the .fasta file
fasta_file = "111_Graph/combined_1.fasta"  

# Create empty lists to store the data
queries = []
sequences = []

# Read through the FASTA file and extract query and sequence information
for record in SeqIO.parse(fasta_file, "fasta"):
    queries.append(record.id)
    sequences.append(str(record.seq))

# Create DataFrame from the lists
fasta_df = pd.DataFrame({'Query': queries, 'Sequence': sequences})

# Display the DataFrame
print(fasta_df)

In [ ]:
distance_df_Sequence=pd.merge(extracted_sequences_Path1_df,fasta_df, on='Query')

In [ ]:
distance_df_Sequence

In [ ]:
distance_df_Sequence.rename(columns = {'Result_node1':'Sequence_GraphAligner'}, inplace = True)
distance_df_Sequence.rename(columns = {'Sequence_y':'Sequence'}, inplace = True)

In [ ]:
distance_df_Sequence

In [ ]:
#Create a method to calculate the levenstein distance
def calculate_edit_distance_sequence_GraphAligner(row):
    return Levenshtein.distance(row['Sequence'], row['Sequence_GraphAligner'])

In [ ]:
distance_df_Sequence['edit_distance_query_sequence_GraphAligner'] = distance_df_Sequence.apply(calculate_edit_distance_sequence_GraphAligner, axis=1)

In [ ]:
distance_df_Sequence